In [1]:
!pip install azureml-train-automl
#다 설치되면 런타임 리셋해야함

In [2]:
import numpy as np
import pandas as pd

from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
import logging

print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings(action='ignore')

pandas version:  0.25.3


In [ ]:
#만들어진 모델로 test셋 평가
df_final = pd.read_csv('/content/drive/Shareddrives/알빠진 포도/데이터 모음집/df2_test_azure3_2_final.csv', encoding='utf-8')   #여기 바꿔줘야함  #클러스터에 해당하는 test파일 불러들이기
df_final = df_final.drop(['Unnamed: 0', 'emd_nm_2term'], axis=1) #불필요 칼럼 제거
df_final['date'] = pd.to_datetime(df_final['date']) #날짜란은 날짜형식으로
test_data = df_final

label='em_g'
test_data['Column2']=np.nan #왠진 모르겠는데 azure평가모델에 column2가 생겨서, test셋에도 이를 만들어야했음

test_labels = test_data['em_g_interpolated'].values
test_labels

#클러스터에 맞는 모델 사용
import pickle 
with open('/content/drive/MyDrive/Colab Notebooks/model_2ago_realfinal3.pkl', 'rb') as f: #여기 바꿔줘야함  #클러스터에 해당하는 pkl형식의 모델로 예측
    autoML = pickle.load(f)

label_query = test_labels.copy().astype(np.float)
label_query.fill(np.nan)
data_trans = autoML.forecast_quantiles(test_data, label_query)
data_trans